In [1]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import shutil

import os
import IPython.display

In [2]:
def load(file_name):
    wav = file_name
    file_sr = librosa.get_samplerate(wav)

    y, sr = librosa.load(wav, sr=file_sr)
    return y, sr

In [ ]:
def checkpoint(y, mininum):
    start = []
    end = []
    first = True
    con = 0
    
    for time in range (len(y)):
        if(abs(y[time]) > mininum):
            if(first):
                first = False
                start_time = time
            else:
                end_time = time
        else:
            con = con + 1
            if(con > 20000):
                con = 0
                if(not first) :
                    start.append(start_time)
                    end.append(end_time)
                    first = True
    return start, end

In [ ]:
def crop(y, sr, start, end, left_padding, right_padding, outputdir):
    for i in range(len(start)):
        height = (end[i]-start[i])/3000
        width = (end[i]-start[i])/500
        if(height > 4) :
            fig = plt.figure(figsize=(width, height))
            sf.write(outputdir + str(i) + ".wav", y[start[i]+left_padding:end[i]+right_padding], sr, subtype='PCM_24')

In [15]:
def make_background(y, sr, outputdir, startpoint=1):
    during = 2 * sr
    amount = int(len(y) / during)
    
    for i in range(amount):
        start = (i*2) * sr
        end = start + during
        newy = y[start:end]
        resample = librosa.resample(newy, sr, 16000)
        sf.write(outputdir + "bg_" + str(i+startpoint) + ".wav", resample, 16000, subtype='PCM_16')

In [21]:
def make_delete_list(input_directory):

    file_list = os.listdir(input_directory)
    delete_list = []
    for file in file_list:
        y, sr = load(input_directory+file)
        sum = 0
        for i in range(sr):
            sum = sum + abs(y[i])
        if(sum < 100):
            delete_list.append(file)
    return delete_list

In [ ]:
y, sr = load('original/non/snake.wav')
start, end = checkpoint(y, 0.0)
print(len(start), len(end))

In [ ]:
crop(y, sr, start, end, 0, 9600, 'split/non/')

In [19]:
y, sr = load('200318.wav')
make_background(y, sr, 'background/bg/', 1631)

In [22]:
delete_list = make_delete_list('background/bg/')
for file in delete_list:
    shutil.move('background/bg/'+file, 'background/background_delete/'+file)

In [ ]:
y, sr = load('background/background_delete/bg_807.wav')
sum = 0
for i in range(sr):
    sum = sum + abs(y[i])
print(sum)

In [ ]:
librosa.display.waveplot(y, sr=sr)

In [13]:
y, sr = load('200318.wav')

In [14]:
sr

44100

In [8]:
y = y[10000000:-1100000]
sf.write("200318.wav", y, sr, subtype='PCM_16')

In [ ]:
IPython.display.Audio(data=y[10000000:11000000], rate=sr)

In [ ]:
IPython.display.Audio(data=y[-1100000:], rate=sr)

In [ ]:
len